In [ ]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

BATCH_SIZE = 32
IMAGE_SIZE = 264
CHANNELS = 3
EPOCHS = 30

# Load Dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Blood cell Cancer [ALL]",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)
class_names = dataset.class_names
print(class_names)

# Split Dataset
train_size = 0.8
train_ds = dataset.take(int(len(dataset) * train_size))
test_ds = dataset.skip(int(len(dataset) * train_size))
val_size = 0.1
val_ds = test_ds.take(int(len(dataset) * val_size))
test_ds = test_ds.skip(int(len(dataset) * val_size))

# Preprocessing
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

resize_and_rescale = layers.Rescaling(1.0/255)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y)).prefetch(buffer_size=tf.data.AUTOTUNE)

# Build CNN + GRU Model
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 4

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Reshape((10, 128)),  # Reshape for GRU (timesteps=10, features=128)
    layers.GRU(64, return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(n_classes, activation='softmax')
])

model.build(input_shape=(BATCH_SIZE,) + input_shape)
model.summary()

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[early_stop])

# Evaluate
scores = model.evaluate(test_ds)
print(f"Test Loss: {scores[0]}, Test Accuracy: {scores[1]}")

# Predictions for Metrics
y_pred = model.predict(test_ds)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Classification Report
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# ROC-AUC
y_true_bin = label_binarize(y_true, classes=[0,1,2,3])
auc_scores = roc_auc_score(y_true_bin, y_pred, multi_class='ovr')
print(f"AUC: {auc_scores}")

# Hyperparameter Tuning (Optional)
# def build_model(hp):
#     model = models.Sequential([
#         resize_and_rescale,
#         layers.Conv2D(hp.Int('conv1', 32, 128, step=32), (3,3), activation='relu', input_shape=input_shape),
#         layers.MaxPooling2D((2,2)),
#         layers.Conv2D(hp.Int('conv2', 64, 256, step=64), (3,3), activation='relu'),
#         layers.MaxPooling2D((2,2)),
#         layers.Flatten(),
#         layers.Dense(hp.Int('dense1', 128, 512, step=128), activation='relu'),
#         layers.Reshape((10, hp.Int('reshape_features', 64, 256, step=64))),
#         layers.GRU(hp.Int('gru_units', 32, 128, step=32), return_sequences=False),
#         layers.Dense(64, activation='relu'),
#         layers.Dropout(0.5),
#         layers.Dense(n_classes, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model
#
# tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=10, directory='tuner_results')
# tuner.search(train_ds, epochs=10, validation_data=val_ds)
# best_model = tuner.get_best_models(1)[0]
# best_model.save("../models/3")

# Save Model
model.save("../models/3")

# Plot History
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()
